In [74]:
# Proceso ETL para cargar archivos CSV a MongoDB

import pandas as pd
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import nltk
from nltk.corpus import stopwords


In [61]:
# Descargar las stopwords en español
nltk.download('stopwords')
spanish_stopwords = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\antho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [62]:
# Configuración de MongoDB
MONGO_URI = "mongodb://localhost:27017/"  # Cambia esto si usas una configuración diferente
DATABASE_NAME = "tesis"
COLLECTION_NAME = "ciberbullying"


In [63]:
# Lista de archivos CSV
# Cambiar por Ruta de archivos CSV
csv_files = [
    "dataset_transformed1.csv",
    "dataset_transformed2.csv",
    "dataset_transformed3.csv",
    "dataset_transformed4.csv",
    "dataset_transformed5.csv"
]

In [64]:
def load_csv_to_mongo(file_list, db_name, collection_name):
    """
    Carga múltiples archivos CSV a una colección de MongoDB.

    Parameters:
        file_list (list): Lista de rutas de archivos CSV.
        db_name (str): Nombre de la base de datos en MongoDB.
        collection_name (str): Nombre de la colección en MongoDB.
    """
    try:
        # Conexión a MongoDB
        client = MongoClient(MONGO_URI)
        db = client[db_name]
        collection = db[collection_name]

        for file in file_list:
            print(f"Procesando archivo: {file}")
            
            # Leer el archivo CSV
            df = pd.read_csv(file)
            
            # Limpiar y convertir a JSON
            data = df.to_dict(orient='records')

            # Insertar datos en MongoDB
            if data:
                collection.insert_many(data)
                print(f"Datos de {file} cargados correctamente.")
            else:
                print(f"El archivo {file} está vacío o no contiene datos válidos.")
        
        print("Todos los archivos se han cargado exitosamente.")
    except Exception as e:
        print(f"Error durante el proceso ETL: {e}")
    finally:
        client.close()

In [65]:
# Ejecutar el ETL
load_csv_to_mongo(csv_files, DATABASE_NAME, COLLECTION_NAME)

Procesando archivo: dataset_transformed1.csv
Datos de dataset_transformed1.csv cargados correctamente.
Procesando archivo: dataset_transformed2.csv
Datos de dataset_transformed2.csv cargados correctamente.
Procesando archivo: dataset_transformed3.csv
Datos de dataset_transformed3.csv cargados correctamente.
Procesando archivo: dataset_transformed4.csv
Datos de dataset_transformed4.csv cargados correctamente.
Procesando archivo: dataset_transformed5.csv
Datos de dataset_transformed5.csv cargados correctamente.
Todos los archivos se han cargado exitosamente.


In [66]:
# --- Análisis de Procesamiento de Lenguaje Natural (PLN) ---

# Cargar datos desde MongoDB
def fetch_data_from_mongo(db_name, collection_name):
    """
    Extrae datos de una colección MongoDB y los convierte en un DataFrame.
    """
    client = MongoClient(MONGO_URI)
    db = client[db_name]
    collection = db[collection_name]
    data = pd.DataFrame(list(collection.find()))
    client.close()
    return data

data = fetch_data_from_mongo(DATABASE_NAME, COLLECTION_NAME)

In [67]:
# Limpieza de datos
def clean_data(data, text_column):
    """
    Limpia el DataFrame eliminando o reemplazando valores nulos en la columna de texto.

    Parameters:
        data (DataFrame): Datos con texto y etiquetas.
        text_column (str): Nombre de la columna de texto.

    Returns:
        DataFrame: Datos limpios.
    """
    data[text_column] = data[text_column].fillna("")  # Reemplazar NaN con cadena vacía
    return data

# Limpieza de los datos antes del preprocesamiento
data = clean_data(data, "texto")

In [68]:
# Preprocesamiento del texto
def preprocess_text(data, text_column, label_columns):
    """
    Preprocesa los textos y divide en conjuntos de entrenamiento y prueba.

    Parameters:
        data (DataFrame): Datos con columnas de texto y etiquetas.
        text_column (str): Nombre de la columna con texto.
        label_columns (list): Nombres de las columnas de etiquetas.

    Returns:
        X_train, X_test, y_train, y_test: Conjuntos de datos divididos.
    """
    vectorizer = TfidfVectorizer(stop_words=spanish_stopwords)
    X = vectorizer.fit_transform(data[text_column])
    y = data[label_columns]
    return train_test_split(X, y, test_size=0.2, random_state=42)

text_column = "texto"  # Columna de texto
label_columns = [
    "insulto", "amenaza", "rumores_difamatorios", "comentarios_humillantes", "lenguaje_discriminatorio",
    "acoso_sexual", "manipulacion_emocional", "lenguaje_vulgar", "acoso_general", "redes_sociales",
    "suicidio_autolesion"
]  # Columnas de etiquetas

In [69]:
# Preprocesamiento
X_train, X_test, y_train, y_test = preprocess_text(data, text_column, label_columns)

In [70]:
# --- Modelo de Machine Learning: Random Forest Multietiqueta ---
rf_base_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model = MultiOutputClassifier(rf_base_model)
rf_model.fit(X_train, y_train)  # Entrena el modelo con los datos de entrenamiento

MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42))

In [71]:
# Predicciones
rf_predictions = rf_model.predict(X_test)

# Evaluación del modelo
print("Reporte de métricas para Random Forest (multietiqueta):")
for i, column in enumerate(label_columns):
    print(f"Etiqueta: {column}")
    print(classification_report(y_test.iloc[:, i], rf_predictions[:, i]))

Reporte de métricas para Random Forest (multietiqueta):
Etiqueta: insulto
              precision    recall  f1-score   support

       False       0.99      1.00      1.00      8615
        True       1.00      0.96      0.98      1347

    accuracy                           0.99      9962
   macro avg       1.00      0.98      0.99      9962
weighted avg       0.99      0.99      0.99      9962

Etiqueta: amenaza
              precision    recall  f1-score   support

       False       0.99      1.00      1.00      9840
        True       1.00      0.54      0.70       122

    accuracy                           0.99      9962
   macro avg       1.00      0.77      0.85      9962
weighted avg       0.99      0.99      0.99      9962

Etiqueta: rumores_difamatorios
              precision    recall  f1-score   support

       False       1.00      1.00      1.00      9369
        True       1.00      0.93      0.96       593

    accuracy                           1.00      9962
   ma

In [75]:
# --- Modelo de Machine Learning: Naive Bayes ---
nb_model = MultiOutputClassifier(MultinomialNB())
nb_model.fit(X_train, y_train)
nb_predictions = nb_model.predict(X_test)

print("\nReporte de métricas para Naive Bayes (multietiqueta):")
for i, column in enumerate(label_columns):
    print(f"\nEtiqueta: {column}")
    print(classification_report(y_test.iloc[:, i], nb_predictions[:, i]))


Reporte de métricas para Naive Bayes (multietiqueta):

Etiqueta: insulto
              precision    recall  f1-score   support

       False       0.88      1.00      0.94      8615
        True       0.97      0.13      0.23      1347

    accuracy                           0.88      9962
   macro avg       0.92      0.56      0.58      9962
weighted avg       0.89      0.88      0.84      9962


Etiqueta: amenaza
              precision    recall  f1-score   support

       False       0.99      1.00      0.99      9840
        True       0.00      0.00      0.00       122

    accuracy                           0.99      9962
   macro avg       0.49      0.50      0.50      9962
weighted avg       0.98      0.99      0.98      9962


Etiqueta: rumores_difamatorios
              precision    recall  f1-score   support

       False       0.94      1.00      0.97      9369
        True       0.70      0.02      0.05       593

    accuracy                           0.94      9962
   

In [76]:
# --- Modelo de Machine Learning: Regresión Logística ---
lr_base_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model = MultiOutputClassifier(lr_base_model)
lr_model.fit(X_train, y_train)
lr_predictions = lr_model.predict(X_test)

print("\nReporte de métricas para Regresión Logística (multietiqueta):")
for i, column in enumerate(label_columns):
    print(f"\nEtiqueta: {column}")
    print(classification_report(y_test.iloc[:, i], lr_predictions[:, i]))



Reporte de métricas para Regresión Logística (multietiqueta):

Etiqueta: insulto
              precision    recall  f1-score   support

       False       0.97      1.00      0.98      8615
        True       1.00      0.77      0.87      1347

    accuracy                           0.97      9962
   macro avg       0.98      0.88      0.93      9962
weighted avg       0.97      0.97      0.97      9962


Etiqueta: amenaza
              precision    recall  f1-score   support

       False       0.99      1.00      0.99      9840
        True       1.00      0.16      0.28       122

    accuracy                           0.99      9962
   macro avg       0.99      0.58      0.64      9962
weighted avg       0.99      0.99      0.99      9962


Etiqueta: rumores_difamatorios
              precision    recall  f1-score   support

       False       0.97      1.00      0.99      9369
        True       1.00      0.52      0.69       593

    accuracy                           0.97      

In [77]:
# --- Modelo de Machine Learning: SVM ---
svm_base_model = SVC(kernel='linear', random_state=42)
svm_model = MultiOutputClassifier(svm_base_model)
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)

print("\nReporte de métricas para SVM (multietiqueta):")
for i, column in enumerate(label_columns):
    print(f"\nEtiqueta: {column}")
    print(classification_report(y_test.iloc[:, i], svm_predictions[:, i]))


Reporte de métricas para SVM (multietiqueta):

Etiqueta: insulto
              precision    recall  f1-score   support

       False       0.99      1.00      1.00      8615
        True       1.00      0.96      0.98      1347

    accuracy                           0.99      9962
   macro avg       1.00      0.98      0.99      9962
weighted avg       0.99      0.99      0.99      9962


Etiqueta: amenaza
              precision    recall  f1-score   support

       False       1.00      1.00      1.00      9840
        True       1.00      0.61      0.76       122

    accuracy                           1.00      9962
   macro avg       1.00      0.80      0.88      9962
weighted avg       1.00      1.00      0.99      9962


Etiqueta: rumores_difamatorios
              precision    recall  f1-score   support

       False       0.99      1.00      1.00      9369
        True       1.00      0.86      0.92       593

    accuracy                           0.99      9962
   macro av

In [72]:
# --- Modelo de Machine Learning: Random Forest ---
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

KeyboardInterrupt: 

In [73]:
# Métricas del modelo
print("Reporte de Random Forest:")
print(classification_report(y_test, rf_predictions))

Reporte de Random Forest:
              precision    recall  f1-score   support

           0       1.00      0.96      0.98      1347
           1       1.00      0.54      0.70       122
           2       1.00      0.93      0.96       593
           3       1.00      0.64      0.78       106
           4       1.00      0.98      0.99      1756
           5       1.00      0.98      0.99      2420
           6       0.00      0.00      0.00         0
           7       1.00      0.96      0.98       941
           8       1.00      0.65      0.78       206
           9       1.00      0.25      0.41        63
          10       1.00      0.03      0.07        29

   micro avg       1.00      0.94      0.97      7583
   macro avg       0.91      0.63      0.69      7583
weighted avg       1.00      0.94      0.96      7583
 samples avg       0.54      0.53      0.53      7583



C:\Users\antho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\antho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\antho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:15